In [12]:
from dask.distributed import Client
import dask.dataframe as dd
import pandas as pd
import numpy as np

In [2]:
client = Client(n_workers=1, threads_per_worker=20, processes=True, memory_limit='10GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 20,Total memory: 7.89 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:65185,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 7.89 GiB
Comm: tcp://127.0.0.1:54173,Total threads: 20
Dashboard: http://127.0.0.1:54174/status,Memory: 7.89 GiB
Nanny: tcp://127.0.0.1:65188,


In [13]:
df = pd.read_csv("https://bitbucket.org/alms/bd-2022/raw/636f07abef5ad88b5287f771bc0fa5359415f1df/tasks/cik.csv")

In [14]:
df.sample(10)

,regin,tik,uik,registered_voters,total_ballots,early_voters,tik_voters,home_voters,empty_ballots,home_ballots,...,missing_ballots,extra_ballots,baburin,grudinin,zhirinovsky,putin,sobchak,suraykin,titov,yavlinsky
50764,Липецкая область,Усманская,УИК №1819,767,775,0,642,89,44,89,...,0,0,0,25,8,683,0,0,3,0
45710,Кировская область,Малмыжская,УИК №666,185,166,0,93,20,53,20,...,0,0,1,7,10,91,0,0,2,0
43183,Калужская область,Спас-Деменская,УИК №2215,449,496,0,264,76,156,76,...,0,0,3,27,21,276,4,5,3,0
50508,Липецкая область,"Липецк, Октябрьская №1",УИК №2257,1650,1400,0,1069,39,292,39,...,0,0,8,142,56,842,13,13,6,10
27704,Приморский край,Яковлевская,УИК №3520,1623,1600,0,1021,95,484,95,...,0,0,9,235,125,693,10,7,8,5
59316,Новосибирская область,Куйбышевская,УИК №610,1200,1100,0,681,40,379,40,...,0,0,4,165,35,499,5,6,2,2
72302,Саратовская область,Балаковская,УИК №563,2482,2250,0,1500,12,738,12,...,0,0,7,210,82,1123,33,11,16,17
70646,Самарская область,Кинель-Черкасская,УИК №1343,327,320,0,229,44,47,44,...,0,0,0,12,11,245,1,2,0,0
87674,город Москва,район Бибирево,УИК №564,2135,1800,0,1173,70,557,70,...,0,0,13,131,49,912,43,15,22,41
54609,Московская область,Щелковская,УИК №3403,1167,1030,0,681,2,347,2,...,0,0,5,86,31,514,14,7,15,10


In [16]:
df = dd.from_pandas(df, npartitions=10)

In [17]:
def compute_presense(dst):
    return ((dst.normal_ballots + dst.bad_ballots) / dst.total_ballots) * 100

In [18]:
pres_df = df.apply(compute_presense, axis=1, meta=(None, 'float64'))

In [19]:
df['presense'] = pres_df

In [20]:
df.groupby(['regin'])

In [21]:
df_sort = df.sort_values('presense', ascending=False)

In [22]:
# Take the candidate and select the UIK where he received the most ballots
df_uik_max_votes = df_sort[df_sort['registered_voters'] > 300]
df_uik_max_votes_putin = df_uik_max_votes['putin'].max()
value = df_uik_max_votes_putin.compute()
df_uik_max_votes[df_uik_max_votes['putin'] == value].compute()

,regin,tik,uik,registered_voters,total_ballots,early_voters,tik_voters,home_voters,empty_ballots,home_ballots,...,extra_ballots,baburin,grudinin,zhirinovsky,putin,sobchak,suraykin,titov,yavlinsky,presense
92828,Территория за пределами РФ,NaN,УИК №8342,7746,13470,4006,3739,1,5719,4007,...,0,22,235,80,7221,82,34,22,29,57.468448


In [23]:
# Count the maximum difference in the region
df_max_by_reg = df_sort.groupby("regin").presense.agg(['min', 'max', 'mean'])
diff = df_max_by_reg['max'] - df_max_by_reg['min']
df_max_by_reg['difference'] = diff
max_diff = df_max_by_reg['difference'].max().compute()
df_max_by_reg[df_max_by_reg['difference'] == max_diff].compute()

,min,max,mean,difference
regin,,,,
Республика Саха (Якутия),0.0,100.0,74.407687,100.0


In [24]:
df_var = df_sort.drop(columns=['registered_voters', 'total_ballots', 'early_voters', 'tik_voters', 'home_voters', 'empty_ballots',
                                        'home_ballots', 'tik_ballots', 'bad_ballots', 'normal_ballots', 'missing_ballots', 'baburin', 'grudinin',
                                        'zhirinovsky', 'putin', 'sobchak', 'suraykin', 'titov', 'yavlinsky', 'extra_ballots'])
df_var_rename = df_var.rename(columns={"presense":"presense_dispersion"})
df_std = df_var_rename.groupby('regin').std()
df_var = df_var_rename.groupby('regin').var()

In [25]:
df_std.compute()

,presense_dispersion
regin,
Алтайский край,10.511163
Амурская область,12.529601
Архангельская область,14.053107
Астраханская область,12.669397
Белгородская область,11.328166
...,...
Чукотский автономный округ,9.564709
Ямало-Ненецкий автономный округ,9.532380
город Москва,8.497469


In [26]:
df_var.compute()

,presense_dispersion
regin,
Алтайский край,110.484558
Амурская область,156.990889
Архангельская область,197.489826
Астраханская область,160.513612
Белгородская область,128.327351
...,...
Чукотский автономный округ,91.483664
Ямало-Ненецкий автономный округ,90.866273
город Москва,72.206972


In [ ]:
tot_ball = df_sort.total_ballots.sum().compute()
df_candidates = df_sort.drop(columns=['regin', 'tik', 'uik', 'registered_voters', 'total_ballots', 'early_voters', 'tik_voters', 'home_voters', 'empty_ballots',
                                        'home_ballots', 'tik_ballots', 'bad_ballots', 'normal_ballots', 'missing_ballots', 'extra_ballots', 'presense'])
candidates_array = df_candidates.to_dask_array()
non_zero_elems = np.count_nonzero(candidates_array, axis=0)
sum = np.sum(candidates_array, axis=0)
divided = np.round(np.divide(sum, tot_ball) * 100).astype(int)

In [28]:
import dask.array as da
names = np.array(['baburin', 'grudinin','zhirinovsky', 'putin', 'sobchak', 'suraykin', 'titov', 'yavlinsky'])
names_arr = da.from_array(names).astype(str)
divided_arr = divided.astype(str)
non_zero_elems_arr = non_zero_elems.astype(str)
# print(type(names_arr))
# names_arr.compute()
cand = dd.concat([dd.from_dask_array(c) for c in [names_arr, divided, non_zero_elems]], axis = 1)
# cand = dd.concat([names_arr_df, divided_df, non_zero_elems_df], axis=1)
cand.compute()
cand.columns = ["candidate_names", "percentage, %", "amount_of_tik"]
cand.reset_index()
cand.set_index("candidate_names")
cand.compute()

,candidate_names,"percentage, %",amount_of_tik
0,baburin,0,74279
1,grudinin,9,92047
2,zhirinovsky,4,90759
3,putin,54,92894
4,sobchak,1,78478
5,suraykin,0,75527
6,titov,1,68858
7,yavlinsky,1,66150
